<a href="https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2005%20-%20LlamaIndex_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q llama-index.post3 deeplake openai cohere

In [1]:
!pip install -q llama-index deeplake openai cohere

In [2]:
from dotenv import load_dotenv

load_dotenv()

import os

print(os.environ['OPENAI_API_KEY'])
os.environ['ACTIVELOOP_TOKEN']

sk-proj-qSr5c57q7fjXHg8921oeT3BlbkFJ0AGUfPTVO1Sk14jx2sDF


'eyJhbGciOiJub25lIiwidHlwIjoiSldUIn0.eyJpZCI6Im1hY2F5YXZlbiIsImFwaV9rZXkiOiJfR05ZV1BTU1Vid0NBSkZFWWRIel9ZOU0tc3YtQm5yejJhbjBnSVNzX0pqVDQifQ.'

In [3]:
import logging
import sys

#You can set the logging level to DEBUG for more verbose output,
# or use level=logging.INFO for less detailed information.
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
from llama_index.core import Settings

print(Settings.embed_model)

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
NumExpr defaulting to 4 threads.


/teamspace/studios/this_studio/.conda/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.9.16) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


ImportError: cannot import name 'Settings' from 'llama_index.core' (/teamspace/studios/this_studio/.conda/lib/python3.10/site-packages/llama_index/core/__init__.py)

# LlamaHub Wikipedia Integration

In [5]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
NumExpr defaulting to 4 threads.
NumExpr defaulting to 4 threads.
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): openaipublic.blob.core.windows.net:443
Starting new HTTPS connection (1): openaipublic.blob.core.windows.net:443
Starting new HTTPS connection (1): openaipublic.blob.core.windows.net:443
DEBUG:urllib3.connectionpool:https://openaipublic.blob.core.windows.net:443 "GET /gpt-2/encodings/main/vocab.bpe HTTP/11" 200 456318
https://openaipublic.blob.core.windows.net:443 "GET /gpt-2/encodings/main/vocab.bpe HTTP/11" 200 456318
https://openaipublic.blob.core.windows.net:443 "GET /gpt-2/encodings/main/vocab.bpe HTTP/11" 200 456318
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): openaipublic.blob.core.windows.net:443
Starting new HTTPS connection (1): openaipublic.blob.core.windows.net:443
Starting new HTTPS connection (1): openaipublic.blob.core.windows.net:443
DEBUG:urllib3.connectionpool:https://op

/teamspace/studios/this_studio/.conda/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.9.16) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): llamahub.ai:443
Starting new HTTPS connection (1): llamahub.ai:443
Starting new HTTPS connection (1): llamahub.ai:443
DEBUG:urllib3.connectionpool:https://llamahub.ai:443 "POST /api/analytics/downloads HTTP/11" 200 63
https://llamahub.ai:443 "POST /api/analytics/downloads HTTP/11" 200 63
https://llamahub.ai:443 "POST /api/analytics/downloads HTTP/11" 200 63


In [7]:
documents = loader.load_data(pages=['Natural Language Processing', 'Artificial Intelligence'])

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): en.wikipedia.org:80
Starting new HTTP connection (1): en.wikipedia.org:80
DEBUG:urllib3.connectionpool:http://en.wikipedia.org:80 "GET /w/api.php?list=search&srprop=&srlimit=1&limit=1&srsearch=Natural+Language+Processing&srinfo=suggestion&format=json&action=query HTTP/11" 301 0
http://en.wikipedia.org:80 "GET /w/api.php?list=search&srprop=&srlimit=1&limit=1&srsearch=Natural+Language+Processing&srinfo=suggestion&format=json&action=query HTTP/11" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): en.wikipedia.org:443
Starting new HTTPS connection (1): en.wikipedia.org:443
DEBUG:urllib3.connectionpool:https://en.wikipedia.org:443 "GET /w/api.php?list=search&srprop=&srlimit=1&limit=1&srsearch=Natural+Language+Processing&srinfo=suggestion&format=json&action=query HTTP/11" 200 174
https://en.wikipedia.org:443 "GET /w/api.php?list=search&srprop=&srlimit=1&limit=1&srsearch=Natural+Language+Processing&srinfo=sugge

In [8]:
len( documents )

2

# Save on DeepLake

In [9]:
from llama_index.vector_stores import DeepLakeVectorStore

my_activeloop_org_id = "macayaven"
my_activeloop_dataset_name = "LlamaIndex_intro"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

# Create an index over the documnts
vector_store = DeepLakeVectorStore(dataset_path=dataset_path, overwrite=False)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): app.activeloop.ai:443
Starting new HTTPS connection (1): app.activeloop.ai:443
DEBUG:urllib3.connectionpool:https://app.activeloop.ai:443 "GET /api/user/profile HTTP/11" 200 None
https://app.activeloop.ai:443 "GET /api/user/profile HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): app.activeloop.ai:443
Starting new HTTPS connection (1): app.activeloop.ai:443
DEBUG:urllib3.connectionpool:https://app.activeloop.ai:443 "GET /api/user/profile HTTP/11" 200 None
https://app.activeloop.ai:443 "GET /api/user/profile HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): spire.bugout.dev:443
Starting new HTTPS connection (1): spire.bugout.dev:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): app.activeloop.ai:443
Starting new HTTPS connection (1): app.activeloop.ai:443
DEBUG:urllib3.connectionpool:https://app.activeloop.ai:443 "GET /api/user/profile HTTP/1

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): app.activeloop.ai:443
Starting new HTTPS connection (1): app.activeloop.ai:443
DEBUG:urllib3.connectionpool:https://app.activeloop.ai:443 "POST /api/event HTTP/11" 200 0
https://app.activeloop.ai:443 "POST /api/event HTTP/11" 200 0


\

DEBUG:botocore.hooks:Event before-parameter-build.s3.GetObject: calling handler <function sse_md5 at 0x7f1ecc4b1ab0>
Event before-parameter-build.s3.GetObject: calling handler <function sse_md5 at 0x7f1ecc4b1ab0>
DEBUG:botocore.hooks:Event before-parameter-build.s3.GetObject: calling handler <function validate_bucket_name at 0x7f1ecc4b1a20>
Event before-parameter-build.s3.GetObject: calling handler <function validate_bucket_name at 0x7f1ecc4b1a20>
DEBUG:botocore.hooks:Event before-parameter-build.s3.GetObject: calling handler <function remove_bucket_from_url_paths_from_model at 0x7f1ecc4b3880>
Event before-parameter-build.s3.GetObject: calling handler <function remove_bucket_from_url_paths_from_model at 0x7f1ecc4b3880>
DEBUG:botocore.hooks:Event before-parameter-build.s3.GetObject: calling handler <bound method S3RegionRedirectorv2.annotate_request_context of <botocore.utils.S3RegionRedirectorv2 object at 0x7f1ebebe1270>>
Event before-parameter-build.s3.GetObject: calling handler <boun

|

...

Sending http request: <AWSPreparedRequest stream_output=False, method=PUT, url=https://s3.us-east-1.wasabisys.com/snark-hub/protected/macayaven/LlamaIndex_intro/dataset_meta.json, headers={'Content-Type': b'application/octet-stream', 'User-Agent': b'Boto3/1.34.131 md/Botocore#1.34.131 ua/2.0 os/linux#5.15.0-1064-aws md/arch#x86_64 lang/python#3.10.14 md/pyimpl#CPython cfg/retry-mode#legacy Botocore/1.34.131', 'Content-MD5': b'7hstfMB8Bl4sbVZoKIgCIg==', 'Expect': b'100-continue', 'X-Amz-Date': b'20240801T003206Z', 'X-Amz-Security-Token': b'4AVHM2iuzr-7GhaTmw_Fcz01vCj5jef14ed2DehZenNSFoHvqt7Qujbk71aHIXBiAFzwWzijnQMN4_sSSAiNqnROzTAUtAaH5uWFVNNJvIoR_zet-c8hZfTIFZyXCj6CwVZutuykRMGouKaqi2mo3GeWGi_kydb4jCEpvYbA8Ho12bEuvebIoXAT2IXGrLz9EG0bn6zC1js6dAXZgYXUZzCjqHAk4eTlqBTwaLij2Zi13yl3aq7QoIfvz5doVLBzayq9Z9vYOvYgBzRwASNaDmk8dXaMRmuOZr1NMahpGMR1B5pPFw4eVPmh6ECgUDRZsEePlnKpn5aLUDrBdbEzYfsgAlqadzR9GAfeXTA34s8uEoTsUn4sz_5SJXoC6l01uXl0-6FRzKgqE0Or_jufzHj0m4TJ0BjjvCuazfm3cGniLebjXWqQ8LLhb0AO3-PmIA4Tj4b

/

DEBUG:botocore.awsrequest:100 Continue response seen, now sending request body.
100 Continue response seen, now sending request body.


\

DEBUG:urllib3.connectionpool:https://s3.us-east-1.wasabisys.com:443 "PUT /snark-hub/protected/macayaven/LlamaIndex_intro/dataset_meta.json HTTP/11" 200 0
https://s3.us-east-1.wasabisys.com:443 "PUT /snark-hub/protected/macayaven/LlamaIndex_intro/dataset_meta.json HTTP/11" 200 0
DEBUG:botocore.parsers:Response headers: {'Date': 'Thu, 01 Aug 2024 00:32:07 GMT', 'ETag': '"ee1b2d7cc07c065e2c6d566828880222"', 'Server': 'WasabiS3/7.20.2951-2024-06-14-7fcea49969 (head13)', 'x-amz-id-2': 'Yc3/LptYWH1s4VjeUNxJN/1wd+sy54uOdFOulDoEYPh+UOB+LFlWBd3EGyi+rFfjUbRs84Fm0kCP', 'x-amz-request-id': '3F5E1803DB69A186:B', 'x-amz-version-id': '001722472327507500073-aYgMquDXYw', 'Content-Length': '0'}
Response headers: {'Date': 'Thu, 01 Aug 2024 00:32:07 GMT', 'ETag': '"ee1b2d7cc07c065e2c6d566828880222"', 'Server': 'WasabiS3/7.20.2951-2024-06-14-7fcea49969 (head13)', 'x-amz-id-2': 'Yc3/LptYWH1s4VjeUNxJN/1wd+sy54uOdFOulDoEYPh+UOB+LFlWBd3EGyi+rFfjUbRs84Fm0kCP', 'x-amz-request-id': '3F5E1803DB69A186:B', 'x-amz-ve

In [10]:
from llama_index.storage.storage_context import StorageContext

storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [11]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): openaipublic.blob.core.windows.net:443
Starting new HTTPS connection (1): openaipublic.blob.core.windows.net:443
DEBUG:urllib3.connectionpool:https://openaipublic.blob.core.windows.net:443 "GET /encodings/cl100k_base.tiktoken HTTP/11" 200 1681126
https://openaipublic.blob.core.windows.net:443 "GET /encodings/cl100k_base.tiktoken HTTP/11" 200 1681126


[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


DEBUG:llama_index.node_parser.node_utils:> Adding chunk: Natural language processing (NLP) is an interdi...
> Adding chunk: Natural language processing (NLP) is an interdi...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: === Statistical NLP (1990s–2010s) ===
Up until ...
> Adding chunk: === Statistical NLP (1990s–2010s) ===
Up until ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: == Approaches: Symbolic, statistical, neural ne...
> Adding chunk: == Approaches: Symbolic, statistical, neural ne...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: === Neural networks ===

A major drawback of st...
> Adding chunk: === Neural networks ===

A major drawback of st...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: === Syntactic analysis ===

Grammar induction
G...
> Adding chunk: === Syntactic analysis ===

Grammar induction
G...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: === Relational semantics (semantics of individu...
> Adding chunk: =

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): spire.bugout.dev:443
Starting new HTTPS connection (1): spire.bugout.dev:443
DEBUG:urllib3.connectionpool:https://spire.bugout.dev:443 "POST /humbug/reports HTTP/11" 200 0
https://spire.bugout.dev:443 "POST /humbug/reports HTTP/11" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): spire.bugout.dev:443
Starting new HTTPS connection (1): spire.bugout.dev:443
DEBUG:urllib3.connectionpool:https://spire.bugout.dev:443 "POST /humbug/reports HTTP/11" 200 0
https://spire.bugout.dev:443 "POST /humbug/reports HTTP/11" 200 0


  0%|          | 0/30 [00:00<?, ?it/s]

...

Certificate path: /teamspace/studios/this_studio/.conda/lib/python3.10/site-packages/certifi/cacert.pem
DEBUG:botocore.awsrequest:Waiting for 100 Continue response.
Waiting for 100 Continue response.
DEBUG:botocore.awsrequest:100 Continue response seen, now sending request body.
100 Continue response seen, now sending request body.
DEBUG:urllib3.connectionpool:https://s3.us-east-1.wasabisys.com:443 "PUT /snark-hub/protected/macayaven/LlamaIndex_intro/embedding/chunks/593917b794064364 HTTP/11" 200 0
https://s3.us-east-1.wasabisys.com:443 "PUT /snark-hub/protected/macayaven/LlamaIndex_intro/embedding/chunks/593917b794064364 HTTP/11" 200 0
DEBUG:botocore.parsers:Response headers: {'Date': 'Thu, 01 Aug 2024 00:42:17 GMT', 'ETag': '"e63934b6deabe4e9327e7cb9a5388a31"', 'Server': 'WasabiS3/7.20.2951-2024-06-14-7fcea49969 (B26-U29)', 'x-amz-id-2': 'k0dYwNg5V7+V1GmlsOXn6dQa1ypYrX5hRrYmwRDcpVTrP7RVN5ROBaGbYNyOHH8T6PF9/RH+yyNW', 'x-amz-request-id': 'C304539ACF3FA49E:B', 'x-amz-version-id': '00172

-

DEBUG:botocore.httpsession:Certificate path: /teamspace/studios/this_studio/.conda/lib/python3.10/site-packages/certifi/cacert.pem
Certificate path: /teamspace/studios/this_studio/.conda/lib/python3.10/site-packages/certifi/cacert.pem
DEBUG:botocore.awsrequest:Waiting for 100 Continue response.
Waiting for 100 Continue response.
DEBUG:botocore.awsrequest:100 Continue response seen, now sending request body.
100 Continue response seen, now sending request body.
DEBUG:urllib3.connectionpool:https://s3.us-east-1.wasabisys.com:443 "PUT /snark-hub/protected/macayaven/LlamaIndex_intro/_embedding_shape/chunks/bf3d04378de24220 HTTP/11" 200 0
https://s3.us-east-1.wasabisys.com:443 "PUT /snark-hub/protected/macayaven/LlamaIndex_intro/_embedding_shape/chunks/bf3d04378de24220 HTTP/11" 200 0
DEBUG:botocore.parsers:Response headers: {'Date': 'Thu, 01 Aug 2024 00:42:17 GMT', 'ETag': '"5d85b73cab9017a6f7780e6e45d3c69d"', 'Server': 'WasabiS3/7.20.2951-2024-06-14-7fcea49969 (B26-U29)', 'x-amz-id-2': 'yf

\

DEBUG:urllib3.connectionpool:https://s3.us-east-1.wasabisys.com:443 "PUT /snark-hub/protected/macayaven/LlamaIndex_intro/id/chunks/cbf251825b24068 HTTP/11" 200 0
https://s3.us-east-1.wasabisys.com:443 "PUT /snark-hub/protected/macayaven/LlamaIndex_intro/id/chunks/cbf251825b24068 HTTP/11" 200 0
DEBUG:botocore.parsers:Response headers: {'Date': 'Thu, 01 Aug 2024 00:42:17 GMT', 'ETag': '"a2ae9c977cf80599763a85685f9fbb47"', 'Server': 'WasabiS3/7.20.2951-2024-06-14-7fcea49969 (B26-U29)', 'x-amz-id-2': 'og4WGhYkzp4XTTibbGyH7BcA2di6LdROeNB/W1rh+eeT6gCLpX2EM0j8gIBcSUeYwDVfJDtsp4zh', 'x-amz-request-id': 'D572543A77B40F12:B', 'x-amz-version-id': '001722472937202010237-WBflLHYTjs', 'x-wasabi-cm-reference-id': '1722472933941 38.27.106.103 ConID:2312929497/EngineConID:29472046/Core:57', 'Content-Length': '0'}
Response headers: {'Date': 'Thu, 01 Aug 2024 00:42:17 GMT', 'ETag': '"a2ae9c977cf80599763a85685f9fbb47"', 'Server': 'WasabiS3/7.20.2951-2024-06-14-7fcea49969 (B26-U29)', 'x-amz-id-2': 'og4WGhY

# Create Nodes

In [12]:
from llama_index.node_parser import SimpleNodeParser

# Assuming documents have already been loaded

# Initialize the parser
parser = SimpleNodeParser.from_defaults(chunk_size=512, chunk_overlap=20)

# Parse documents into nodes
nodes = parser.get_nodes_from_documents(documents)
print( len( nodes ) )

DEBUG:llama_index.node_parser.node_utils:> Adding chunk: Natural language processing (NLP) is an interdi...
> Adding chunk: Natural language processing (NLP) is an interdi...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: When the "patient" exceeded the very small know...
> Adding chunk: When the "patient" exceeded the very small know...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: === Statistical NLP (1990s–2010s) ===
Up until ...
> Adding chunk: === Statistical NLP (1990s–2010s) ===
Up until ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: === Neural NLP (present) ===
In 2003, word n-gr...
> Adding chunk: === Neural NLP (present) ===
In 2003, word n-gr...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: == Approaches: Symbolic, statistical, neural ne...
> Adding chunk: == Approaches: Symbolic, statistical, neural ne...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: === Neural networks ===

A major drawback of st...
> Adding chunk: =

48

# Create index from Documents

In [13]:
from llama_index import GPTVectorStoreIndex

index = GPTVectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()
response = query_engine.query("What does NLP stands for?")
response.response

DEBUG:llama_index.node_parser.node_utils:> Adding chunk: Natural language processing (NLP) is an interdi...
> Adding chunk: Natural language processing (NLP) is an interdi...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: === Statistical NLP (1990s–2010s) ===
Up until ...
> Adding chunk: === Statistical NLP (1990s–2010s) ===
Up until ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: == Approaches: Symbolic, statistical, neural ne...
> Adding chunk: == Approaches: Symbolic, statistical, neural ne...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: === Neural networks ===

A major drawback of st...
> Adding chunk: === Neural networks ===

A major drawback of st...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: === Syntactic analysis ===

Grammar induction
G...
> Adding chunk: === Syntactic analysis ===

Grammar induction
G...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: === Relational semantics (semantics of individu...
> Adding chunk: =

DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Thu, 01 Aug 2024 00:44:20 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-allow-origin', b'*'), (b'openai-model', b'text-embedding-ada-002'), (b'openai-organization', b'user-avu13tqik3o0kfeyfplcf4id'), (b'openai-processing-ms', b'82'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15552000; includeSubDomains; preload'), (b'x-ratelimit-limit-requests', b'5000'), (b'x-ratelimit-limit-tokens', b'1000000'), (b'x-ratelimit-remaining-requests', b'4999'), (b'x-ratelimit-remaining-tokens', b'990238'), (b'x-ratelimit-reset-requests', b'12ms'), (b'x-ratelimit-reset-tokens', b'585ms'), (b'x-request-id', b'req_e712b5e99387bec4f5d63a31ada24be7'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Set-Cookie', b'__cf_bm=XxRKoZwldAUDBdpeRotfxFUXWXcd_zhbobmi0r9h_VE-1722473060-1.0.1.1-_WMIWqNJsNY

'NLP stands for Natural Language Processing.'

# Environment

In [14]:
!pip list


Package                 Version
----------------------- ------------
aioboto3                13.1.1
aiobotocore             2.13.1
aiofiles                24.1.0
aiohttp                 3.9.5
aioitertools            0.11.0
aiosignal               1.3.1
anyio                   4.4.0
asttokens               2.4.1
async-timeout           4.0.3
attrs                   23.2.0
backoff                 2.2.1
beautifulsoup4          4.12.3
boto3                   1.34.131
botocore                1.34.131
certifi                 2024.7.4
charset-normalizer      3.3.2
click                   8.1.7
cohere                  4.37
comm                    0.2.2
cssselect               1.2.0
dataclasses-json        0.5.14
debugpy                 1.6.7
decorator               5.1.1
deeplake                3.8.8
Deprecated              1.2.14
dill                    0.3.8
distro                  1.9.0
exceptiongroup          1.2.2
executing               2.0.1
fastavro                1.9.5
feedfinder2    